# HW8 Question 2

## Summary Stats

In [33]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math

pd.options.mode.chained_assignment = None  # default='warn' ## Silence warnings

LTCM_Returns = pd.read_excel("ltcm_exhibits_data.xlsx",
                        sheet_name = "Exhibit 2",
                        skiprows = 2,
                        index_col = 0)

LTCM_Returns = LTCM_Returns.dropna(axis=0, how="any")

## LTCM Returns row are good now

SPX = pd.read_excel("gmo_analysis_data.xlsx",
                        sheet_name = "returns (total)",
                        index_col = 0)


## Copied from HW 6                     
def Presenter(input_data, y_col_name):
    temp_annualized_col = input_data[y_col_name] * 12
    ## Yeah Yeah Yeah I can write some
    ## Ultra-Fancy math stuff proving this and that for annualizing
    ## the vol and skewness and whatever
    ## But time are better spent elsewhere.

    mean = temp_annualized_col.mean()
    vol = temp_annualized_col.std()
    sharpe = mean / vol
    skew = temp_annualized_col.skew()
    ex_kurt = temp_annualized_col.kurt() - 3
    Hist_VaR = temp_annualized_col.quantile(0.05)
    return pd.DataFrame({
        "Mean": mean,
        "Volatility": vol,
        "Sharpe Ratio": sharpe,
        "Skewness": skew,
        "Ex Kurtosis": ex_kurt,
        "Historical VaR": Hist_VaR
    }, index = [f'summary {y_col_name}'])

result = pd.concat([
    Presenter(LTCM_Returns, y_col_name="Gross Monthly Performancea"),
    Presenter(LTCM_Returns, y_col_name='Net Monthly Performanceb'),
    Presenter(SPX, y_col_name="SPY")
    ], axis=0)

result

,Mean,Volatility,Sharpe Ratio,Skewness,Ex Kurtosis,Historical VaR
summary Gross Monthly Performancea,0.293887,0.472344,0.622187,-0.296428,-1.430646,-0.316800
summary Net Monthly Performanceb,0.207170,0.387647,0.534429,-0.817870,-0.094463,-0.268800
summary SPY,0.103263,0.516319,0.199997,-0.573558,-2.035350,-0.865156


## Comment Q2.1 NEED MORE

Well, I think about 1/3 of the return are lost in gross to net, though it also decreased the volatility and bring more to negative terriroty. The investment are doing much better than SPY anyways.

## Q2.2

Everything Below and Before Q3 are in MONTHLY return, NOT ANNUALIZED, unless otherwise stated.

In [53]:
## The regression is on market excess returns.


LTCM_Returns = pd.read_excel("ltcm_exhibits_data.xlsx",
                        sheet_name = "Exhibit 2",
                        skiprows = 2,
                        index_col = 0)

LTCM_Returns = LTCM_Returns.dropna(axis=0, how="any")

## LTCM Returns row are good now

SPX = pd.read_excel("gmo_analysis_data.xlsx",
                        sheet_name = "returns (total)",
                        index_col = 0)

LTCM_Returns.index = pd.to_datetime(LTCM_Returns.index)
SPX.index = pd.to_datetime(SPX.index) + pd.Timedelta(days=1) ## The data are from the end of month. Gotta add 1 day

Dataset_for_234 = pd.merge(LTCM_Returns[['Net Monthly Performanceb']],
                        SPX[['SPY']], left_index = True, right_index = True)

Dataset_for_234 = Dataset_for_234.rename(columns={'Net Monthly Performanceb': 'Rm_LTCM'})

model = sm.OLS(Dataset_for_234['Rm_LTCM'], 
            sm.add_constant(Dataset_for_234['SPY'])).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Rm_LTCM   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.1350
Date:                Sat, 25 Nov 2023   Prob (F-statistic):              0.715
Time:                        23:59:10   Log-Likelihood:                 107.30
No. Observations:                  53   AIC:                            -210.6
Df Residuals:                      51   BIC:                            -206.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0163      0.005      3.172      0.0

### Report

Alpha is 0.0163, Beta is 0.0505, R^2 is 0.003. The insignificance of the result is astoundingly significant.

## Q2.3

In [55]:
Dataset_for_234['SPY_Sq'] = Dataset_for_234['SPY'] ** 2


model = sm.OLS(Dataset_for_234['Rm_LTCM'], 
            sm.add_constant(Dataset_for_234[['SPY', 'SPY_Sq']])).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Rm_LTCM   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9642
Date:                Sun, 26 Nov 2023   Prob (F-statistic):              0.388
Time:                        00:02:17   Log-Likelihood:                 108.23
No. Observations:                  53   AIC:                            -210.5
Df Residuals:                      50   BIC:                            -204.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0211      0.006      3.388      0.0

### Report

We have increased R^2 tenfold to 0.037, or 3.7%. We still see a positive beta for market return, which means that the fund is long market. 

## Q2.4

In [60]:
Dataset_for_234['SPY_Call_Like_K1'] = Dataset_for_234['SPY'].apply(lambda x: max(x - .03, 0))
Dataset_for_234['SPY_Put_Like_K2'] = Dataset_for_234['SPY'].apply(lambda x: max(-.03 - x, 0))

model = sm.OLS(Dataset_for_234['Rm_LTCM'], 
            sm.add_constant(Dataset_for_234[['SPY', 'SPY_Call_Like_K1', 'SPY_Put_Like_K2']])).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Rm_LTCM   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.172
Date:                Sun, 26 Nov 2023   Prob (F-statistic):              0.330
Time:                        00:08:24   Log-Likelihood:                 109.06
No. Observations:                  53   AIC:                            -210.1
Df Residuals:                      49   BIC:                            -202.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0154      0.006  

# HW 8 Question 3


## Static Carry Table

In [111]:




## Read Data
RF_rates = pd.read_excel("fx_carry_data.xlsx",
                        sheet_name = "risk-free rates",
                        index_col = 0)

Log_RF_rates = RF_rates.map(lambda x: math.log(1+float(x)))

FX_rates = pd.read_excel("fx_carry_data.xlsx",
                        sheet_name = "fx rates",
                        index_col = 0)

Log_FX_rates = FX_rates.map(lambda x: math.log(float(x)))

Log_Raw_DF = pd.merge(Log_RF_rates, Log_FX_rates, left_index=True, right_index=True)

## Helper
def currency_name_converter(input):
    Dict1 = {
        "UK": "GBP",
        "EU": "EUR",
        "SZ": "CHF",
        "JP": "JPY",
    }
    if len(input) == 2:
        try:
            return Dict1[input]
        except:
            return "Not Found!"
    elif len(input) == 3:
        for key, value in Dict1.items():
            if value == input:
                return key
        return "Not Found!"
    else:
        return "Not Found!"

def data_extractor(currency_name):
    ## Use 3-Letter currency_name
    FX_Name = f'US{currency_name_converter(currency_name)}'
    Rate_Name = f'{currency_name}1M'
    Temp_Dataset = Log_Raw_DF[['USD1M', Rate_Name, FX_Name]]

    Temp_Dataset[f'{FX_Name}_t_plus_one'] = Temp_Dataset.loc[:, FX_Name].shift(-1) ## t+1 stuff
    return Temp_Dataset

def excess_return_finder(currency_name):
    FX_Name = f'US{currency_name_converter(currency_name)}'
    Rate_Name = f'{currency_name}1M'
    Temp_Dataset = data_extractor(currency_name)
    ## Excessive Return at t+1
    Temp_Dataset[f'Log_Ex_Return_{currency_name}_t_plus_one'] = \
        Temp_Dataset[f'{FX_Name}_t_plus_one'] - Temp_Dataset[FX_Name] + Temp_Dataset[Rate_Name] - Temp_Dataset['USD1M']

    ## Back to t
    Temp_Dataset[f'Log_Ex_Return_{currency_name}'] = Temp_Dataset[f'Log_Ex_Return_{currency_name}_t_plus_one'].shift(1) 
    return Temp_Dataset[f'Log_Ex_Return_{currency_name}'] 

currency_list = ['GBP', 'EUR', 'CHF', 'JPY']
Result_cols = []

for i in currency_list:
    Result_cols.append(f'Log_Ex_Return_{i}')
    Log_Raw_DF[f'Log_Ex_Return_{i}'] = excess_return_finder(i)


Temp_for_report = Log_Raw_DF[Result_cols]
Temp_for_report = Temp_for_report.dropna(axis=0).map(lambda x: x * 12)


summary_df = pd.DataFrame({
    'Mean': Temp_for_report.mean(),
    'Vol': Temp_for_report.std()
})

summary_df['Sharpe'] = summary_df['Mean'] / summary_df['Vol']

print(summary_df)

                       Mean       Vol    Sharpe
Log_Ex_Return_GBP -0.003502  0.298961 -0.011713
Log_Ex_Return_EUR -0.004351  0.328097 -0.013263
Log_Ex_Return_CHF  0.004312  0.342103  0.012604
Log_Ex_Return_JPY -0.017415  0.316936 -0.054947


# Q3 Part 1 Explanations

The mean log excessive returns are generally zero, and the variance (for log returns) are around 0.3. It seems that JPY has a better average excessive return, but it is still not much.

# Q3 Part 2

Let us write everything in Log form here unless otherwise specified

## (a)

The Uncovered Interest Parity essentially states that the expected value of excessive returns should be zero. Because $E(r^~) = E(s_{t+1}) - s_t -r^{f, FX}_{t, t+1} - r^{f, \$}_{t, t+1} = 0$.

The conclusion is only partly verified. For instance, the worst performer of JPY at log return -0.017415 should give us a negative return of 1.726%, which is not a small number. Others, however, gives an excessive return of +- 0.4%, which is not big but also have some value.

In addition, we are seeing very low Sharpe ratio and very high variance here. The UIP is generally held, but in short term it can be violated due to high variance.

## (b)

Long the CHF, or the Swiss Francs.

## (c)

The JPY gives a negative excessive return.

# Q3 Part 3

In [112]:
## Let us use our old functions

def Single_Currency_Regression(currency_name):
    ## 3 digit currency name
    Temp_Dataset = data_extractor(currency_name)
    FX_Name = f'US{currency_name_converter(currency_name)}'
    Rate_Name = f'{currency_name}1M'
    Temp_Dataset[f'{FX_Name}_Diff'] = Temp_Dataset[f'{FX_Name}_t_plus_one'] - Temp_Dataset[FX_Name]
    Temp_Dataset[f'{Rate_Name}_USD_Rate_Diff'] = Temp_Dataset['USD1M'] - Temp_Dataset[Rate_Name]
    Temp_Dataset = Temp_Dataset.dropna(axis = 0)
    model = sm.OLS(Temp_Dataset[f'{FX_Name}_Diff'], 
                sm.add_constant(Temp_Dataset[f'{Rate_Name}_USD_Rate_Diff'])).fit()
    ## print(model.summary())
    return [model.params['const'], ## Alpha
            model.params[f'{Rate_Name}_USD_Rate_Diff'], ## Beta
            model.rsquared
            ]

Result_DF = {}

for i in currency_list:
    Result_DF[i] = Single_Currency_Regression(i)

Result_DF = pd.DataFrame(Result_DF, index=['Alpha', 'Beta', 'R Squared'])
print(Result_DF)

                GBP       EUR       CHF       JPY
Alpha     -0.000489  0.000586  0.003631 -0.000500
Beta       0.485836 -1.256358 -1.646596  0.371473
R Squared  0.000382  0.002610  0.003948  0.000501


## Q3 Part 3 (b)

Well, a relative increase in foreign risk-free rate should reflect in an increase in the X-variable of the regression. Therefore, we conclude the following:

(a) A strengthening of USD means that US dollar appreciates, therefore driving down the value of $s_[t+1]$. This means a decrease in Y-variables of the regression. It happens in EUR (Euros) and CHF (Swiss Francs) due to their negative betas

(b) Similarly, weakening of the USD means a higher value of $s_{t+1}$, which leads to an increase in Y-variables. This happens in GBP (British Pounds) and JPY (Japanese Yen), due to their negative betas

(c) The predictability is strongest for CHF due to highest R^2 value.


In [113]:
Result_DF

,GBP,EUR,CHF,JPY
Alpha,-0.000489,0.000586,0.003631,-0.000500
Beta,0.485836,-1.256358,-1.646596,0.371473
R Squared,0.000382,0.002610,0.003948,0.000501


In [114]:
Result_DF.loc['Alpha', 'GBP']

-0.0004890240195433706

# Q3 Part 4

In [115]:
def carry_trade_finder(currency_name):
    Rate_Name = f'{currency_name}1M'
    return Result_DF.loc['Alpha', currency_name] + (Result_DF.loc['Beta', currency_name] - 1) * (Log_Raw_DF['USD1M'] - Log_Raw_DF[Rate_Name])

carry_trade_cols = []
for i in currency_list:
    name = f'{i}_carry_trade_exp_log_gain'
    carry_trade_cols.append(name)
    Log_Raw_DF[name] = carry_trade_finder(i)

## We now have the result
print("Carry Trade Exp Gain:")
print(Log_Raw_DF[carry_trade_cols].head())

print("Number of Positive Entries:")
print(Log_Raw_DF[carry_trade_cols].apply(lambda x: (x > 0).sum()))

Carry Trade Exp Gain:
            GBP_carry_trade_exp_log_gain  EUR_carry_trade_exp_log_gain  \
DATE                                                                     
1999-01-31                     -0.000068                     -0.002826   
1999-02-28                     -0.000239                     -0.002867   
1999-03-31                     -0.000325                     -0.003063   
1999-04-30                     -0.000326                     -0.003787   
1999-05-31                     -0.000335                     -0.003866   

            CHF_carry_trade_exp_log_gain  JPY_carry_trade_exp_log_gain  
DATE                                                                    
1999-01-31                     -0.004606                     -0.002871  
1999-02-28                     -0.004550                     -0.002948  
1999-03-31                     -0.004609                     -0.002991  
1999-04-30                     -0.005039                     -0.003000  
1999-05-31           